In [ ]:
## Project Cardcaptor 


In [27]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [77]:
origin_df = pd.read_csv("ssg_card.csv", encoding = 'utf-8-sig')
#origin_df = pd.DataFrame()
origin_df.fillna("", inplace = True)

In [78]:
url = 'https://www.ssg.com/event/eventMain.ssg' 
r = requests.get(url)

if r.status_code == 200:
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')

else : 
    print(r.status_code)

# import requests
# from bs4 import BeautifulSoup

# headers = {'User-Agent': 'Mozilla/5.0'}

# url = "https://linkedin.com/company/1005"

# r = requests.get(url, headers=headers)
# print(r.text)

# soup = BeautifulSoup(r.text, 'html.parser')
# print(soup.prettify())

In [79]:
events = soup.find_all(class_ = 'evt_osmu_unit' )
event_ls = [] 
for event in events : 
    try : 
        name = event.find(class_ = 'eo_tit').find("strong").text
        if "카드" in name : 
            event_ls.append(event)
    except : 
        continue
        #하단 문화이벤트 영역이 공란일 경우 에러 발생시키기 때문에 try except 처리한다 

# [event for event in events if "카드" in event.find(class_ = 'eo_tit').find("strong").text]

In [84]:
df = pd.DataFrame( {"date_start" : []
                    ,"date_end" : [] 
                    ,"card" : []
                    ,"descript" : []
                    ,"platform" : []
                    ,"detail_date" : []
                    ,"detail_card" : []
                    ,"detail_desc" : []
                    ,"detail_goods" : []
                    ,"detail_limit" : []
                    ,"update_dt" : []
                    })

In [85]:
update_dt = datetime.today()
print(update_dt)

2022-09-25 22:01:23.608581


In [86]:

main_url = 'https://www.ssg.com'
update_dt = datetime.today()

for event in event_ls : 
    
    #기본정보
    date_total = event.find(class_ = 'eo_period').text.split(" - ")
    card = event.find(class_ = 'eo_tit').find("strong").text
    descript = event.find(class_ = 'desc1').text + " " + event.find(class_ = 'desc2').text
    platform = event.find(class_ = 'eo_mall').text.replace("\n", "")
    
    #링크타고 들어가서 얻는 디테일 
    #아니 왜 링크 유형이 제각각이야 
    event_url = event.find(href=True)["href"]
    if "https://" not in event_url : 
        second_url = main_url + event_url
    else : 
        second_url = event.find(href=True)["href"]
        
    r2 = requests.get(second_url)

    if r2.status_code == 200:
        html = r2.text
        soup = BeautifulSoup(html, 'html.parser')

    else : 
        print(r2.status_code)
        
    try : 
        details = soup.find(class_="ecard_info").find_all("li")
        for d in details : 

            item = d.find(class_="tit").text
            content = d.text.replace(d.find(class_="tit").text, "")

            if item == '행사기간' : 
                detail_date = content
            elif item == '대상카드' : 
                detail_card = content
            elif item == '행사내용' :
                detail_desc = content
            elif item == '대상상품' : 
                detail_goods = content
            else : 
                detail_limit = content
    except : 
        detail_date, detail_card, detail_desc, detail_goods, detail_limit = "", "", "", "", ""

    row = pd.DataFrame.from_dict( [{"date_start" : date_total[0]
                        ,"date_end" : date_total[1]
                        ,"card" : card
                        ,"descript" : descript
                        ,"platform" : platform
                        ,"detail_date" : detail_date
                        ,"detail_card" : detail_card
                        ,"detail_desc" : detail_desc
                        ,"detail_goods" : detail_goods
                        ,"detail_limit" : detail_limit
                        ,"update_dt" : update_dt}])
    df = pd.concat([df, row], ignore_index = True)

In [87]:
df.fillna("", inplace = True)

In [88]:
print(len(df))

14


In [89]:
df.detail_date[6]

''

In [90]:
df.tail()

,date_start,date_end,card,descript,platform,detail_date,detail_card,detail_desc,detail_goods,detail_limit,update_dt
9,2022.09.24,2022.09.25,롯데카드,SSGPAY로 7만원 이상 결제 시 5% 청구할인,ssg.com,2022. 9. 24 (토) ~ 9. 25 (일) (※승인일시 기준),"롯데카드 (※단, 법인/기프트/체크카드 제외)",SSGPAY로 롯데카드 7만원 이상 결제 시 5% 청구할인,SSG.COM상품 (※서비스 상품/상품권 및 일부상품 제외),1인 1일 최대 5만원 한도,2022-09-25 22:01:24.675675
10,2022.09.24,2022.09.25,SSG.COM카드 Edition2,최대 3만원! 5% 즉시할인,ssg.com,2022. 9. 24 (토) ~ 9. 25 (일) (※승인일시 기준),"SSG.COM카드 Edition2 (※선불,체크,기프트,법인카드 제외)",SSG.COM카드 Edition2로 결제 시 대상상품 금액의 5% 즉시할인,"SSG.COM상품 (※서비스 상품, 상품권 및 일부상품 제외)",결제페이지에서 SSG.COM카드 Edition2를 선택하시면 즉시할인이 적용 된 카...,2022-09-25 22:01:24.675675
11,2022.09.19,2022.09.25,신한카드,대상상품 10% 청구할인,shinsegae mall,2022. 9. 19 (월) ~ 9. 25 (일) (※승인일시 기준),"신한카드 (※단, 신한BC/법인/선불/기프트 카드 제외, 체크카드 포함)",구찌/버버리/프리미엄아울렛/까사미아 상품 신한카드로 10만원 이상 결제 시 10% ...,구찌/버버리/프리미엄아울렛/까사미아 상품 (※일부상품 제외),구찌/버버리/프리미엄아울렛/까사미아 상품 행사 상품을 함께 결제 할 경우 청구할인 ...,2022-09-25 22:01:24.675675
12,2022.07.08,2023.06.28,스마일클럽 전용 카드,스마일클럽 가입비 100% 지원 SSG머니 최대 12% 적립,ssg.com,,,,,,2022-09-25 22:01:24.675675
13,2022.03.28,2023.02.16,현대카드,장기무이자 최대 20개월 혜택!,ssg.com,,,,,,2022-09-25 22:01:24.675675


In [91]:
print(len(df))
print(len(origin_df))
df = pd.concat([df, origin_df], ignore_index = True)
print(len(df))


14
33
47


In [92]:
#df = pd.concat([df, origin_df], ignore_index = True)
df = df.drop_duplicates(subset = ["date_start", "date_end", "card", "descript", "platform", "detail_date", "detail_card", "detail_desc", "detail_goods", "detail_limit"], keep = "last")

In [95]:
len(df)

31

In [94]:
df.to_csv("test.csv", index = False, encoding = 'utf-8-sig')

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 16 to 46
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date_start    31 non-null     object
 1   date_end      31 non-null     object
 2   card          31 non-null     object
 3   descript      31 non-null     object
 4   platform      31 non-null     object
 5   detail_date   31 non-null     object
 6   detail_card   31 non-null     object
 7   detail_desc   31 non-null     object
 8   detail_goods  31 non-null     object
 9   detail_limit  31 non-null     object
 10  update_dt     31 non-null     object
dtypes: object(11)
memory usage: 2.9+ KB
